# Exemplo: Exercício sugerido para estimação de parâmetros.

## Introdução

O presente exemplo aborda o ajuste de modelos para os dados experimentais apresentados na Tabela 1. Faz-se necessário escolher entre dois modelos, para avaliar qual melhor se adequa à estes dados:

$y = \left(\frac{α_1\cdot x}{1+β_1\cdot x}\right)$          (1)

$y = \left(α_2\cdot x^{β_2}\right)$                           (2)

em que $α_1$, $α_2$, $β_1$ e $β_2$ são os parâmetros a serem estimados para cada modelo. Este exemplo é proposto por Schwaab e Pinto [1].


**Tabela 1: Dados experimentais. Fonte: [1]**


Variáveis|  1 |  2 |  3 |  4 |  5  |  6  |  7  |  8  |  9  |  10 |
:------:|:--:|:--:|:--:|:--:|:---:|:---:|:---:|:---:|:---:|:---:|
$x^e$   |1.00|2.00|3.00|5.00|10.00|15.00|20.00|30.00|40.00|50.00|
$y^e$   |1.66|6.07|7.55|9.72|15.24|18.79|19.33|22.38|24.27|25.51|


Então, o problema de otimização a ser resolvido envolve o uso da função objetivo de mínimos quadrados ponderados, conforme:

$F_{Obj} = \sum_{i=1}^{NE} \left({\frac{(y^{e}_i-y_i^{m})^2}{σ^2_{i}}}\right)$ (3)

sujeito a (1) ou (2).

Para resolução deste problema no MT_PEU serão usados os símbolos:

* Símbolos das grandezas independentes: $x_1$, $x_2$
* Símbolos das grandezas dependentes: $y_1$, $y_1$
* Símbolos dos parâmetros: $α_1$, $α_2$, $β_1$, $β_2$

O subíndice 1 refere-se ao modelo (1), e o subíndice 2 refere-se ao modelo (2).

## Importação de pacotes

Importação de bibliotecas (pacotes) necessárias para execução do código.

* **MT_PEU**: biblioteca que contém as principais funcionalidades da ferramenta

    * Importar a classe **EstimacaoNaoLinear**, que será utilizada neste exemplo de estimação não linear

* **casadi**: biblioteca para computação simbólica

    * será necessária somente a função **vertcat** para construção do modelo

In [1]:
from MT_PEU import EstimacaoNaoLinear
from casadi import vertcat # função para concatenar expressões do modelo

## Criação de modelo

Os modelos (1) e (2) representam o comportamento da grandeza dependente, no qual os parâmetros $α_1$, $α_2$, $β_1$ e $β_2$ serão estimados.

Este modelo é definido, então, na forma de uma subrotina, **def**, do python e representado por:

In [2]:
def Modelo(param,x,args):
    
    a1, b1, a2, b2 = param[0], param[1], param[2], param[3]
    x1, x2 = x[0], x[1]

    return vertcat(a1*x1/(1+b1*x1), a2*(x2**b2))

## Inicialização de classe

Para executar a estimação, o primeiro passo é configurar a classe **EstimacaoNaoLinear**, inicialmente, incluindo informações básicas:

* O modelo,
* Os símbolos para as grandeza independentes *(x)*; 
* Os símbolos para as grandezas dependentes *(y)*; 
* Os símbolos para os parâmetros *(param)*;
* O nome do projeto, o nome da pasta onde serão gerados os resultados.

In [3]:
Estime = EstimacaoNaoLinear(Modelo,simbolos_x=['x1','x2'],simbolos_y=['y1','y2'],simbolos_param=['alpha1','alpha2', 'beta1', 'beta2'],
                          label_latex_param=[r'$\alpha_1$',r'$\alpha_2$',r'$\beta_1$',r'$\beta_2$'],unidades_y=['kg','kg'],Folder='Exemplo4')

## Inclusão de dados

Os dados experimentais disponibilizados em [2] das variáveis dependente ($y_1$ e $y_2$) e das variáveis independentes ($x_1$ e $x_2$), são apresentados abaixo na forma de listas:

In [4]:
x1 = [1.,2.,3.,5.,10,15.,20.,30.,40.,50.]
y1 = [1.66,6.07,7.55,9.72,15.24,18.79,19.33,22.38,24.27,25.51]
x2 = [1.,2.,3.,5.,10,15.,20.,30.,40.,50.]
y2 = [1.66,6.07,7.55,9.72,15.24,18.79,19.33,22.38,24.27,25.51]


Como entrada **obrigatória**, a plataforma MT_PEU necessita da **incerteza dos dados experimentais** (ux1, ux2, uy1).
Neste exemplo, foram adotados o valor 1 para as incertezas.

In [5]:
ux1 = [1]*10
ux2 = [1]*10
uy1 = [1]*10
uy2 = [1]*10

**Inclusão de dados experimentais na estimação:**

O método *setDados* é utilizado para incluir os dados para as grandezas dependentes e independentes. Sintaxe:

* identificação da grandeza, se independente ou dependente: 0 ou 1 (respectivamente)
* os dados experimentais e suas incertezas inseridos em sequência na forma de tuplas.

In [6]:
Estime.setDados(0,(x1,ux1),(x2,ux2))
Estime.setDados(1, (y1, uy1), (y2, uy2))

Define que os dados experimentais previamente inseridos serão utilizados como um conjunto de dados para o qual os 
parâmetros serão estimados:

In [7]:
Estime.setConjunto(tipo='estimacao')

## Otimização

Neste exemplo, o usuário tem a opção de escolha do algoritmo a ser utilizado na otimização. 
Disponiveis: 'ipopt', 'bonmin'. 
Caso opte por não escolher, será utilizado o algoritmo default: ipopt.

Para realizar a otimização é obrigatório informar a estimativa inicial que, neste caso, será [3, 0.1, 5,0.4].

In [8]:
Estime.optimize(initial_estimative=[3, 0.1, 5,0.4],algoritmo='ipopt')


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



## Incerteza


Neste exemplo é possível escolher o método útilizado para avaliar a incerteza 
Métodos disponíveis: 2InvHessiana, Geral, SensibilidadeModelo. 
Por definição o preenchimeto da região de verossimilhança é 'True', caso necessário esta opção pode ser alterada.

Caso o usuário **não** tenha interesse em **avaliar a incerteza dos parâmetros** para o **cálculo da predição**, apenas **não** executar **Estime.incertezaParametros**.


In [9]:
Estime.incertezaParametros(metodoIncerteza='SensibilidadeModelo',preencherregiao=False)

## Predição de análise de resíduos

 No método predição, é feita a avaliação da grandeza dependente com base nos parâmetros fornecidos. 
 A covariância é avaliada, e consequentemente a eficiencia do modelo. 
 Em análise de residuos é possível verificar possíveis relações de dependencia e/ou tendencia entre as variaveis. 
 Testes estatisticos como o de homocedasticidade, $\chi^2$, dentre outros são realizados
 nesta etapa. A analise de residuos é feita prioritariamente com os dados de validação.

In [10]:
Estime.predicao()
Estime.analiseResiduos()

## Gráficos e relatório

Nesta etapa ocorre a geração dos dados de saída do programa : relátorios e gráficos. 
Os gráficos são gerados de acordo com as etapas que foram realizadas. No relátorio contém informações a respeito.
dos testes estatisticos, função objetivo, matriz de covariância, status da otimização, dentre outros.

In [11]:
etapas = ['otimizacao','grandezas-entrada', 'predicao','grandezas-calculadas','analiseResiduos', 'regiaoAbrangencia']
Estime.graficos(etapas)
Estime.relatorio()

/home/giufba-admin/Documentos/MT_PEU/MT_PEU.py:2018: UserWarning: Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.
  warn('Os gráficos de otimizacao não puderam ser criados, o algoritmo de otimização utilizado não possui gráficos de desempenho OU o método otimizacao não foi executado.',UserWarning)
/home/giufba-admin/Documentos/MT_PEU/Graficos.py:392: RuntimeWarning: divide by zero encountered in double_scalars
  alpha  = [vecs[1,0]/vecs[0,0],vecs[1,1]/vecs[0,1]]
/home/giufba-admin/Documentos/MT_PEU/Graficos.py:393: RuntimeWarning: invalid value encountered in double_scalars
  lamb   = [sqrt(c2/(invcov[0,0]+2*alpha_i*invcov[0,1] + alpha_i**2*invcov[1,1])) for alpha_i in alpha]


<Figure size 432x288 with 0 Axes>

<Figure size 360x240 with 0 Axes>

## Referências: 

[1] SRINIVASAN, R.; LEVI, A. A.. Kinetics of the Thermal Isomerization of Bicyclo [2.1.1 ]hexane. Journal Of The American Chemical Society, [s.l.], v. 85, n. 21, p.3363-3365, 5 nov. 1963. American Chemical Society (ACS)

[2] SCHWAAB, M.M.;PINTO, J.C. Análise de Dados Experimentais I: Fundamentos da Estátistica e Estimação de Parâmetros. 
Rio de Janeiro: e-papers, 2007.

INMETRO.: Avaliação de dados de medição — Guia para a expressão de incerteza de medição. Rio de Janeiro: Jcgm, 2008.